## scrape spreads 
http://www.footballlocks.com/nfl_point_spreads.shtml

In [2]:
import os
import pandas
import urllib2
import requests
import bs4

In [3]:
url = 'http://www.footballlocks.com/nfl_point_spreads.shtml'
r  = requests.get(url)
data = r.text

soup = bs4.BeautifulSoup(data, "lxml")

# get the tables w/ the spreads
tt = soup.findAll("table", {"width" : "580"})

In [6]:
df_spreads = pandas.DataFrame()
for i in range(2):  # this probably needs to change once the preseason is over
    dfs = pandas.read_html(str(tt[i]), )
    df_spreads = df_spreads.append(dfs[0])

df_spreads.index = range(len(df_spreads))
df_spreads.columns = ['date', 'favorite', 'spread', 'underdog']    
filter_bad = df_spreads.favorite == 'Favorite'

df_spreads = df_spreads[~filter_bad] 

# get the home favorite
df_spreads['home_favorite'] = df_spreads.favorite.str.contains('^At ')
# fix any spreads that are tied (PK)
df_spreads.loc[df_spreads.spread.astype(unicode).str.contains('PK'), 'spread'] = .1

# flip sign on spread for away favorite
df_spreads['factor'] = 1
df_spreads.loc[df_spreads.home_favorite == True,'factor'] = -1
df_spreads['spreads2'] = df_spreads.spread.astype(float) * df_spreads.factor

# get the home team
df_spreads['home_team'] = df_spreads.favorite
home_filter = df_spreads.underdog.str.contains('^At ') | df_spreads.underdog.str.contains('\(at ')
df_spreads.loc[home_filter, 'home_team'] = df_spreads.loc[home_filter, 'underdog']
df_spreads.home_team = df_spreads.home_team.str.replace('^At ', '')
df_spreads['datetime'] = pandas.to_datetime('2016/'+df_spreads.date.str.split(" ", expand=True)[0], format='%Y/%m/%d').dt.date
df_spreads

,date,favorite,spread,underdog,home_favorite,factor,spreads2,home_team,datetime
1,10/20 8:25 ET,At Green Bay,-7.5,Chicago,True,-1,7.5,Green Bay,2016-10-20
2,10/23 9:30 ET,NY Giants,-3,Los Angeles(at London),False,1,-3.0,Los Angeles(at London),2016-10-23
3,10/23 1:00 ET,Minnesota,-2.5,At Philadelphia,False,1,-2.5,Philadelphia,2016-10-23
4,10/23 1:00 ET,At Kansas City,-6.5,New Orleans,True,-1,6.5,Kansas City,2016-10-23
5,10/23 1:00 ET,At Detroit,-1,Washington,True,-1,1.0,Detroit,2016-10-23
6,10/23 1:00 ET,At Cincinnati,-10,Cleveland,True,-1,10.0,Cincinnati,2016-10-23
7,10/23 1:00 ET,Buffalo,-3,At Miami,False,1,-3.0,Miami,2016-10-23
8,10/23 1:00 ET,At Jacksonville,-1,Oakland,True,-1,1.0,Jacksonville,2016-10-23
9,10/23 1:00 ET,At Tennessee,-2.5,Indianapolis,True,-1,2.5,Tennessee,2016-10-23
10,10/23 1:00 ET,At NY Jets,-1,Baltimore,True,-1,1.0,NY Jets,2016-10-23


In [11]:
# stupid fix for London game
df_spreads.home_team[2] = 'Los Angeles'

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
# read in the master lines file
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
path_to_lines = codeDir + "data/lines/"
lines_file = path_to_lines + "nflAllLines.csv"

df_lines = pandas.read_csv(lines_file)
df_lines.Date = pandas.to_datetime(df_lines.Date)
df_lines

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,week
0,1978-09-02,New York Giants,19.0,Tampa Bay Buccaneers,13.0,2.0,,1978,NaN
1,1978-09-03,Green Bay Packers,13.0,Detroit Lions,7.0,8.0,,1978,NaN
2,1978-09-03,Houston Oilers,14.0,Atlanta Falcons,20.0,-3.5,,1978,NaN
3,1978-09-03,Kansas City Chiefs,24.0,Cincinnati Bengals,23.0,8.0,,1978,NaN
4,1978-09-03,Miami Dolphins,20.0,New York Jets,33.0,-4.0,,1978,NaN
5,1978-09-03,Oakland Raiders,6.0,Denver Broncos,14.0,-4.0,,1978,NaN
6,1978-09-03,Pittsburgh Steelers,28.0,Buffalo Bills,17.0,-10.5,,1978,NaN
7,1978-09-03,Los Angeles Rams,16.0,Philadelphia Eagles,14.0,-6.0,,1978,NaN
8,1978-09-03,San Diego Chargers,24.0,Seattle Seahawks,20.0,-6.0,,1978,NaN
9,1978-09-03,San Francisco 49ers,7.0,Cleveland Browns,24.0,4.5,,1978,NaN


In [13]:
# find the right week/game and update the spread
week_filter = (df_lines.Date <= df_spreads.datetime.max()) & (df_lines.Date >= df_spreads.datetime.min())

for ii,rr in df_spreads.iterrows():
    print ii, rr['home_team'], rr['spreads2']
    if 'NY' in rr['home_team']:
        rr['home_team'] = rr['home_team'].split(' ')[1]
    game_filter = df_lines[week_filter]['Home Team'].str.contains(rr['home_team'])
    irow = df_lines[week_filter][game_filter].index[0]
    df_lines.loc[irow, 'Line'] = rr['spreads2']

1 Green Bay 7.5
2 Los Angeles -3.0
3 Philadelphia -2.5
4 Kansas City 6.5
5 Detroit 1.0
6 Cincinnati 10.0
7 Miami -3.0
8 Jacksonville 1.0
9 Tennessee 2.5
10 NY Jets 1.0
11 Atlanta 6.5
12 San Francisco -2.0
13 Pittsburgh -7.0
14 Arizona 1.5
15 Denver 7.5


In [14]:
lines_file = path_to_lines + "nflAllLines.csv"
df_lines.to_csv(lines_file, index=False)